In [2340]:
from plotly.offline import init_notebook_mode, iplot

import pandas as pd
import plotly.graph_objects as go
import math
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import random
from decimal import Decimal

In [2341]:

def to_lonlat(point_xy, origin):
    r = 6371000

    o_lat = math.radians(Decimal(origin[0]))
    o_lon = math.radians(Decimal(origin[1]))
    cos_phi = math.cos(o_lat)
    lat = Decimal(point_xy[1])/Decimal(r) + Decimal(o_lat)
    lon = Decimal(point_xy[0])/Decimal((r*cos_phi)) + Decimal(o_lon)
    return {'lon': math.degrees(lon), 'lat': math.degrees(lat)}


def seg_lonlat(seg):
    return [to_lonlat(i,origin) for i in seg]

def plot_segment(fig,seg):

    df = pd.DataFrame(seg)
    fig.add_trace(go.Scattermapbox(
        mode = "markers+lines",
        lon = df['lon'],
        lat = df['lat'],
        marker = {'size': 10}))

def wrap(x, low, high):
    if (low <= x and x < high):
        return x

    range = high - low
    inv_range = 1.0/range
    num_wraps = math.floor((x - low) * inv_range)
    return x - (range * num_wraps)


def waypoint_from_heading_and_distance(lat_traffic, lon_traffic, bearing, dist):
    r = 6371000
    bearing = wrap(bearing, 0, 2*math.pi)
    radius_ratio = dist / r

    lat_traffic_rad = math.radians(lat_traffic)
    lon_traffic_rad = math.radians(lon_traffic)

    lat_end = math.asin(math.sin(lat_traffic_rad) * math.cos(radius_ratio) +
                        math.cos(lat_traffic_rad) * math.sin(radius_ratio) * math.cos(bearing))

    lon_end = lon_traffic_rad + math.atan2(math.sin(bearing) * math.sin(radius_ratio) * math.cos(
        lat_traffic_rad), math.cos(radius_ratio) - math.sin(lat_traffic_rad) * math.sin(lat_end))

    lat_end = math.degrees(lat_end)
    lon_end = math.degrees(lon_end)

    return lat_end, lon_end, bearing, radius_ratio


def get_bearing_to_next_waypoint(lat_start, lon_start, lat_end, lon_end):

    lat_start_rad = math.radians(lat_start)
    lat_end_rad = math.radians(lat_end)

    cos_lat_end = math.cos(lat_end_rad)
    d_lon = math.radians(lon_end - lon_start)

    y = (math.sin(d_lon) * cos_lat_end)
    x = (math.cos(lat_start_rad) * math.sin(lat_end_rad) -
         math.sin(lat_start_rad) * cos_lat_end * math.cos(d_lon))

    return wrap(math.atan2(y, x), -math.pi, math.pi)


def get_distance_to_next_waypoint(lat_uav, lon_uav, lat_end, lon_end):

    r = 6371000

    lat_uav_rad = math.radians(lat_uav)
    lat_end_rad = math.radians(lat_end)

    d_lat = lat_end_rad - lat_uav_rad
    d_lon = math.radians(lon_end) - math.radians(lon_uav)

    a = math.sin(d_lat / 2.0) * math.sin(d_lat / 2.0) + math.sin(d_lon / 2.0) * math.sin(d_lon / 2.0) * math.cos(lat_uav_rad) * math.cos(
        lat_end_rad)

    c = math.atan2(math.sqrt(a), math.sqrt(1.0 - a))

    return r * 2.0 * c


def get_distance_to_line(lat_uav, lon_uav, lat_traffic, lon_traffic, lat_end, lon_end):

    return_value = 1
    crosstrack_distance = 0.0

    dist_to_end = get_distance_to_next_waypoint(
        lat_uav, lon_uav, lat_end, lon_end)

    if (dist_to_end < 0.1):
        return_value = 0 

    bearing_end = get_bearing_to_next_waypoint(
        lat_uav, lon_uav, lat_end, lon_end)
    bearing_track = get_bearing_to_next_waypoint(
        lat_traffic, lon_traffic, lat_end, lon_end)
    bearing_diff = wrap(bearing_track - bearing_end, -math.pi, math.pi)

    if (bearing_diff > math.pi/2 or bearing_diff < -math.pi/2):
        return_value = 0

    crosstrack_distance = (dist_to_end) * math.sin(bearing_diff)

    return return_value, crosstrack_distance 


In [2342]:
#uav location 
origin=[32.617013,-96.490564]
h = 0 #x
k = 0 #y
uav_z = 1000

vx_now = 0
vy_now = 0
vz_now = 0

crosstrack_separation = 500
vertical_separation = 500
collision_time_threshold = 60.0

rot_step = 1.86



In [2343]:
lines=[
    [[10000,10000],[h,k]],
    [[1000,-1000],[h,k]],
    [[-5000,-4000],[h,k]],
    [[-3000,3000],[h,k]],
    [[-200,200],[h,k]],
    [[0,0],[h,k]],
    [[50,100],[h,k]],
    [[1000000,9000000],[h,k]],
    [[800,30],[h,k]],
    [[7000,3],[h,k]],
]

fig = go.Figure()
    
[plot_segment(fig,seg_lonlat(i)) for i in lines]

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': origin[1], 'lat': origin[0]},
        'style': "stamen-terrain",
        'zoom': 13})

fig.write_html('traffic.html', auto_open=True)

In [2344]:

dist_xy = []

for i in range(len(lines)):
    r = math.sqrt(lines[i][0][0]*lines[i][0][0] + lines[i][0][1]*lines[i][0][1])
    dist_xy.append(r)  

In [2345]:
a = [seg_lonlat(i) for i in lines]

lat = []
lon = []
address = []

for i in range(0,len(a)):
    #print(str(a[i][0]['lat'])+","+str(a[i][0]['lon']))
    lat.append(a[i][0]['lat'])
    lon.append(a[i][0]['lon'])
    address.append(random.randint(1000,9999))


In [2364]:
print(address)

[6751, 2839, 2431, 3222, 2171, 7666, 4249, 8438, 4110, 5798]


In [2346]:
heading = np.arange(-math.pi, math.pi, math.radians(rot_step)) 

In [2347]:
headings = []

ct_dist = []
ct_distance_conflict_threshold = []

vertical_arr = []
vertical_check = []

dist_xyz = []
traffic_vel = []
time_check = []
collision_times = []

In [2348]:
for j in range(len(dist_xy)):

    prediction_dist = dist_xy[j] + 1000     

    for i in heading:

        headings.append(i)       

        lat_end, lon_end, bearing, radius_ratio = waypoint_from_heading_and_distance(lat[j],lon[j],i,prediction_dist)
        

        return_value, ct  = get_distance_to_line(origin[0],origin[1],lat[j],lon[j],lat_end, lon_end)

        ct = abs(ct)

        #print(lat_end,lon_end, bearing, radius_ratio, return_value, ct)

        ct_dist.append(ct)
        if(ct < crosstrack_separation and return_value):
            ct_distance_conflict_threshold.append(True)
        else:
            ct_distance_conflict_threshold.append(False)

        vertical_separation_check = random.choice([True, False])
        if (vertical_separation_check):
            z_diff = random.randrange(0, vertical_separation-1)
            vertical_check.append(True)
        else:
            z_diff = random.randrange(vertical_separation+1, 1000)
            vertical_check.append(False)
            
        sign = random.choice([-1, 1])
        vertical_arr.append(sign*z_diff+uav_z)

        xyz = math.sqrt(z_diff*z_diff + dist_xy[j]*dist_xy[j])
        dist_xyz.append(xyz)

        collision_time_check = random.choice([True, False])
        if (collision_time_check):
            time_to_collision = random.randrange(1, collision_time_threshold)
            time_check.append(True)
            collision_times.append(time_to_collision)
        else:
            time_to_collision = random.randrange(collision_time_threshold+1, 10*collision_time_threshold)
            time_check.append(False)
            collision_times.append(time_to_collision)

        vel = xyz/time_to_collision
        traffic_vel.append(math.sqrt((vel*vel)/2))

        

In [2349]:
lat_traffic = np.array([])

for i in lat:
    for j in range(len(heading)):
        lat_now = np.array([i])
        lat_traffic = np.concatenate([lat_traffic, lat_now])


lon_traffic = np.array([])

for i in lon:
    for j in range(len(heading)):
        lon_now = np.array([i])
        lon_traffic = np.concatenate([lon_traffic, lon_now])


In [2350]:
in_conflict = []

for i in range(len(time_check)):
    if((time_check[i]== True) and (vertical_check[i] == True) and (ct_distance_conflict_threshold[i] == True)):
        in_conflict.append(True)
    else:
        in_conflict.append(False)

In [2351]:
#print(vertical_check)

In [2352]:
print(vertical_arr[78])

946


In [2353]:
#print(headings)

In [2354]:
print(ct_dist[78])

13903.21572374496


In [2355]:
#print(ct_distance_conflict_threshold)

In [2356]:
print(time_check[78])

False


In [2357]:
collision_times[78]

456

In [2358]:
traffic_vel[78]

21.929984429241845

In [2359]:
print(in_conflict[78])

False


In [2360]:
print(np.count_nonzero(in_conflict))

158


In [2361]:
print("dist_xy", len(dist_xy))
print("lat_traffic", len(lat_traffic))
print("lon_traffic", len(lon_traffic))



dist_xy 10
lat_traffic 1940
lon_traffic 1940


In [2362]:
"""      	
	struct traffic_data_s {
	double lat_traffic
	double lon_traffic
	float alt_traffic
	float heading_traffic
	float vxy_traffic
	float vz_traffic
	bool in_conflict
"""

traffic_data = np.zeros(shape=(7, len(lat_traffic)))

traffic_data = np.concatenate([traffic_data,[lat_traffic]])

traffic_data = np.concatenate([traffic_data,[lon_traffic]])


alt_traffic = np.asarray(vertical_arr)
traffic_data = np.concatenate([traffic_data,[alt_traffic]])

headings_arr = np.asarray(headings)
traffic_data = np.concatenate([traffic_data,[headings_arr]])

vxy_traffic = np.asarray(traffic_vel)
traffic_data = np.concatenate([traffic_data,[vxy_traffic]])

vz_traffic = np.asarray(traffic_vel)
traffic_data = np.concatenate([traffic_data,[vz_traffic]])

in_conflict_arr = np.asarray(in_conflict)
traffic_data = np.concatenate([traffic_data,[in_conflict_arr]])

for i in range(7):
	traffic_data = np.delete(traffic_data,0,0)



In [2363]:
np.savetxt("foo.csv", traffic_data.transpose(), delimiter=",",newline='}, \n {')
print(traffic_data.transpose().shape)

(1940, 7)
